<a href="https://colab.research.google.com/github/davidzeng21/ddls-2024/blob/main/ddls_2024_module_6_computer_lab_automating_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 6: Automating Scientific Discovery
---

Welcome to the practical session of [Data-Driven Life Sciences course module 6](https://ddls.aicell.io/course/ddls-2024/module-6/), created by Gabriel Reder. And, Professor Wei Ouyang, teaching assistants Songtao Cheng, and Nils Mechtel made some modifications.

## Introduction
This notebook will guide exploration of automating the experimental side of scientific discovery in the laboratory. The aim is to encourage thinking about everyday manual tasks in experiments that can be aided through the use of LLM models and AI systems boosting the power of laboratory robotics.

**Authors**: Gabriel Reder (gk@reder.io)


### Important Note for This Lab Notebook:

- **🌞 Tasks Introduction:** Sections marked with a 🌞 symbol introduce an exercise or question. Please read these sections carefully to understand the concepts and tasks involved.

- **⭐ Your Answer Here:** Cells marked with a ⭐ symbol indicate where you need to write your answer. Please provide your code or answer there.

## Installing dependencies
We'll need the `opentrons` python package to simulate running protocols, so we'll install that now.

In [3]:
!pip install opentrons

# Creating robotic lab protocols

## Opentron liquid handlers
For this exercise, we'll suppose you are running a lab equipped with an Opentron flex liquid handling robot:

<figure>
<center>
<img src="https://cdn11.bigcommerce.com/s-zmwgbev7rb/products/255/images/814/Flex-6__84033.1719413544.386.513.png?c=1" width="350">
</figure>

If you'd like to see the robot in action, take a look at this marketing [video](https://www.youtube.com/watch?v=d6ln-0LeT8A)

## A sample Opentron protocol

One thing that's nice about Opentrons is that they have a Python API, meaning we can write scripts to execute protocols directly on the robot. Take a look at the sample protocol below. Can you tell what's going on?

In [4]:
%%writefile sample_protocol.py

from opentrons import protocol_api

requirements = {"robotType": "Flex", "apiLevel":"2.19"}

def run(protocol: protocol_api.ProtocolContext):
    plate = protocol.load_labware(
        load_name="corning_96_wellplate_360ul_flat",
        location="D1")
    tiprack_1 = protocol.load_labware(
        load_name="opentrons_flex_96_tiprack_200ul",
        location="D2")
    trash = protocol.load_trash_bin("A3")
    pipette = protocol.load_instrument(
        instrument_name="flex_1channel_1000",
        mount="left",
    tip_racks=[tiprack_1])

    pipette.pick_up_tip()
    pipette.aspirate(100, plate["A1"])
    pipette.dispense(100, plate["B1"])
    pipette.drop_tip()

Overwriting sample_protocol.py


🌞 <font color='orange'>**Exercises**:</font>

**Note** - Do these exercises manually, do *not* use ChatGPT yet.

1. Watch the opentron protocol designer [video](https://support.opentrons.com/s/article/How-to-write-a-basic-protocol-in-Protocol-Designer) to familiarize yourself with the robot and the environment. We're going to use the Python API directly rather than the `Protocol Designer` GUI. After watching the video, what do you think the protocol above is doing? Write a step-by-step plain text protocol that corresponds to what you think the protocol does.

2. What do the `location` designations (e.g. `D1` and `D2`) in the script correspond to?

3. Now familiarize yourself with the opentron [API](https://docs.opentrons.com/v2/) and pay special attention to the [Hardware Modules](https://docs.opentrons.com/v2/new_modules.html#) and the [Labware Library](https://labware.opentrons.com/). What is the difference between `Hardware` and `Labware` in the context of the opentron API?

4. Find the API names for the following pieces of hardware/labware:
  - Thermocycler module
  - Magnetic block
  - GEB 96 Tip Rack 10 $\mu$L
  - Thermo Scientific Nunc 96 Well Plate 1300 $\mu$L



---
⭐ Double click to write down your answers here


```
Answer:

1.
  1. Define the robot, pipettes, and modules.
  2. Define liquids.
  3. Define deck setup.
    Starting Deck State - allows us to see the state of a deck before any steps are performed.
    Add Step - allows us to add liquid handling steps to the protocol.
    Final Deck State - displays the state of the deck after all steps are performed.
  4. Liquid transfer.
    Move Labware - moves labware on or off the deck of the robot.
    Transfer - moves liquid between labware.
    Mix - mixes liquid in a specific well.
    Pause - temporarily stops the robot for a specified amount of time and then resumes the protocol.
  5. File export.
  6. Demo of robot running.


2. It corresponds to the slot on the deck.

3. Hardware is the fixed settlement of the machine.
   Labware is the one-time used items in the automatic process.

4.
    1. "thermocyclerModuleV2"
    2. "magneticBlockV1"
    3. "geb_96_tiprack_10ul"
    4. "thermoscientificnunc_96_wellplate_1300ul"

```
---

## Simulating protocols

The opentron API allows us to simulate protocols, even if we don't have a robot. Let's try this out on our sample protocol. Make sure you have executed the cell above containing the sample protocol. This will save the protocol to a colab file called `sample_protocol.py`

Now, execute the cell below to simulate the sample protocol and see what it does.

In [5]:
!opentrons_simulate sample_protocol.py

/root/.opentrons/robot_settings.json not found. Loading defaults
from_conf None default {<InstrumentProbeType.PRIMARY: 1>: '/data/pressure_sensor_data.csv'}
Belt calibration not found.
Picking up tip from A1 of Opentrons Flex 96 Tip Rack 200 µL on slot D2
Aspirating 100.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on slot D1 at 716.0 uL/sec
Dispensing 100.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on slot D1 at 716.0 uL/sec
Dropping tip into Trash Bin on slot A3


🌞 <font color='orange'>**Exercises**:</font>

You've been given a 96-well plate where column 1 contains 300$\mu$L of sample in each well of the column. You want to perform a serial dilution at 1:10 dilution factors across each column. So column 1 will contain the sample, column 2 will contain the sample diluted at 1:10, column 3 will be another 1:10 dilution (so 1:100 from the original sample) etc. You have the following working parameters for the protocol:

  - You are diluting the sample into water. You have a reservoir of water available to you on the opentron.
  - You want the final dilution column to be a 1:1000000 dilution from the original sample.
  - Use version 2.19 of the opentron API
  - Use an 8 channel pipette.
  - You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
  - Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.


Use a combination of ChatGPT and the opentron API documentation to write a protocol that will perform this serial dilution on an opentron Flex using version 2.19 of the API. The newest ChatGPT should be somewhat familiar with the opentron API and should be able to write opentron python code. However, you may want to give it an example script to give hints about the specific structure and syntax you are looking to generate. You may also have to revise your script by hand using a combination of ChatGPT and the API documentation.

The protocol must simulate succesfully.

⭐ Write your answer in the code cell below and use the simulation cell below that to try the protocol

In [6]:
%%writefile serial_dilution.py

from opentrons import protocol_api

# Define metadata and requirements
requirements = {"robotType": "Flex", "apiLevel": "2.19"}

def run(protocol: protocol_api.ProtocolContext):
    # ⭐ Load labware
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 'D1')  # 96-well plate in slot 1
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 'D2')      # Tip rack in slot 2
    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 'D3')        # Reservoir with water in slot 3
    trash = protocol.load_trash_bin("A3")  # Use default trash

    # ⭐ Load pipette
    pipette = protocol.load_instrument('flex_8channel_1000', mount='left', tip_racks=[tiprack])

    # ⭐ Variables
    dilution_factor = 10
    num_of_dilutions = 6
    sample_volume = 30  # μL of sample to transfer from one column to the next
    diluent_volume = 270  # μL of water in each column

    # Pickup the pipette tips
    pipette.pick_up_tip()

    # Add diluent to columns 2 through 7 (270 µL of water in each well across all rows)
    for col in range(2, num_of_dilutions + 2):  # Column 2 to 7
        pipette.distribute(
            diluent_volume,
            reservoir['A1'],  # Assuming water is in well A1 of the reservoir
            [plate.columns_by_name()[str(col)][0]],  # Distribute to all rows in the column
            new_tip='never'
        )

    # Perform serial dilution across all rows in columns 1 to 6
    for col in range(1, num_of_dilutions + 1):  # Columns 1 to 6
        pipette.transfer(
            sample_volume,
            plate.columns_by_name()[str(col)][0],  # Source column
            plate.columns_by_name()[str(col+1)][0],  # Destination column
            mix_after=(3, 50),  # Mix after transferring to ensure proper dilution
            new_tip='never'  # Reuse the same tips
        )

    # ⭐ Optional: Dispose of remaining sample from the last column to avoid overflow
    pipette.blow_out(plate.columns_by_name()[str(num_of_dilutions+1)][0])

    # Dispose of the tips
    pipette.drop_tip(trash)


Writing serial_dilution.py


In [7]:
!opentrons_simulate serial_dilution.py

/root/.opentrons/robot_settings.json not found. Loading defaults
from_conf None default {<InstrumentProbeType.PRIMARY: 1>: '/data/pressure_sensor_data.csv'}
Belt calibration not found.
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on slot D2
Logs from this command:
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
Distributing 270.0 from A1 of NEST 12 Well Reservoir 15 mL on slot D3 to A2 of Corning 96 Well Plate 360 µL Flat on slot D1
	Transferring 270.0 from A1 of NEST 12 Well Reservoir 15 mL on slot D3 to A2 of Corning 96 Well Plate 360 µL Flat on slot D1
		Aspirating 275.0 uL from A1 of NEST 12 Well Reservoir 15 mL on slot D3 at 716.0 uL/sec
		Dispensing 270.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on slot D1 at 716.0

# Automating a real-life protocol

Now we're going to explore how we can use the combination of lab robotics and LLMs to (partly) reproduce a real-world protocol.

Our protocol will come from this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/) testing the effects of various antifungal compounds on the formation of yeast biofilms.

We will see how we can speed the process of experimentation using the tools we've been learning about.

Watch the entire protocol [video](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/bin/jove-44-2287-pmcvs_normal.mp4) to understand the purpose of the protocol and the steps involved.

🌞 <font color='orange'>**Exercise**:</font>

Say you've performed the protocol by hand up until the point of setting up the plate with yeast for biofilm formation (2:49 in the video). You now have your yeast culture prepared and a 96-well plate ready to use.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to ChatGPT to create a script that will perform the protocol steps in the video outlined from 2:49 to 3:06 using an opentron Flex. You can make the following assumptions:

- You have prepared 80mL of diluted yeast sample and have transferred it to an appropriate reservoir.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.

Simulate the protocol and make sure it runs correctly.

⭐ Write your answer in the code cell below and use the simulation cell below that to try the protocol

In [28]:
%%writefile yeast_plating.py

from opentrons import protocol_api

# Metadata and requirements
requirements = {"robotType": "Flex", "apiLevel": "2.19"}

def run(protocol: protocol_api.ProtocolContext):
    # ⭐ Load labware
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 'D1')  # 96-well plate in slot 1
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 'D2')      # Tip rack in slot 2
    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 'D3')        # Reservoir with yeast culture in slot 3
    trash = protocol.load_trash_bin("A3")   # Trash for used tips

    # ⭐ Load pipette
    pipette = protocol.load_instrument('flex_8channel_1000', mount='left', tip_racks=[tiprack])

    # ⭐ Volume to transfer
    transfer_volume = 100  # µL of yeast suspension to transfer

    # Pick up the pipette tips
    pipette.pick_up_tip()

    # Transfer yeast sample to each well in columns 1 through 11 (excluding column 12)
    for col in range(1, 12):  # Columns 1 to 11
        pipette.transfer(
            transfer_volume,
            reservoir['A1'],  # Assuming yeast suspension is in well A1 of the reservoir
            plate.columns_by_name()[str(col)],  # Transfer to all wells in the column
            new_tip='never'  # Reuse the same tip
        )

    # Drop the pipette tips
    pipette.drop_tip(trash)



Overwriting yeast_plating.py


In [29]:
!opentrons_simulate yeast_plating.py

/root/.opentrons/robot_settings.json not found. Loading defaults
from_conf None default {<InstrumentProbeType.PRIMARY: 1>: '/data/pressure_sensor_data.csv'}
Belt calibration not found.
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on slot D2
Logs from this command:
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
Transferring 100.0 from A1 of NEST 12 Well Reservoir 15 mL on slot D3 to A1 of Corning 96 Well Plate 360 µL Flat on slot D1
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on slot D3 at 716.0 uL/sec
	Dispensing 100.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on slot D1 at 716.0 uL/sec
Transferring 100.0 from A1 of NEST 12 Well Reservoir 15 mL on slot D3 to A2 of Corning 96 Well Plate 360 µL Flat on slo

🌞 <font color='orange'>**Exercise**:</font>

Now say you've continued the protocol, performing the incubation and washes by hand. You have prepared a stock solution of 1,024 $\mu$g/mL fluconazole and a stock solution of buffered RPMI 1640 medium.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to ChatGPT to create a script that will perform the protocol steps for Antifungal Susceptibility Testing of Biofilms (3:57 to 5:04). You can make the following assumptions:

- You have prepared 80mL of fluconazole and 80mL of media and have transferred them to appropriate reservoirs.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use a *different* set of pipette tips for (1) dispensing the antifungal (2) dispensing the medium (3) performing the dilutions. So you should use a total of three set of tips (pick up / drop off operations) over the course of the protocol.

Simulate the protocol and make sure it runs correctly.

In [34]:
%%writefile antifungal_testing.py

from opentrons import protocol_api

# Metadata and requirements
requirements = {"robotType": "Flex", "apiLevel": "2.19"}

def run(protocol: protocol_api.ProtocolContext):
    # ⭐ Load labware
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 'B1')  # 96-well plate in slot 1
    tiprack_antifungal = protocol.load_labware('opentrons_96_tiprack_300ul', 'B2')  # Tip rack 1 for antifungal
    tiprack_media = protocol.load_labware('opentrons_96_tiprack_300ul', 'B3')       # Tip rack 2 for media
    tiprack_serial = protocol.load_labware('opentrons_96_tiprack_300ul', 'D1')      # Tip rack 3 for serial dilution
    fluconazole_reservoir = protocol.load_labware('nest_12_reservoir_15ml', 'D2')   # Fluconazole in slot 5
    media_reservoir = protocol.load_labware('nest_12_reservoir_15ml', 'D3')         # RPMI 1640 in slot 6
    trash = protocol.load_trash_bin("A3") # Trash for used tips

    # ⭐ Load pipette
    pipette = protocol.load_instrument('flex_8channel_1000', mount='left', tip_racks=[tiprack_antifungal, tiprack_media, tiprack_serial])

    # ⭐ Volume definitions
    antifungal_volume = 200  # µL for fluconazole
    media_volume = 100       # µL for RPMI 1640
    transfer_volume = 100    # µL for serial dilution transfer

    # Step 1: Add 200 µL of fluconazole to wells in column 1
    pipette.pick_up_tip(tiprack_antifungal.wells()[0])
    pipette.transfer(
        antifungal_volume,
        fluconazole_reservoir['A1'],
        plate.columns_by_name()['1'],  # Transfer to all wells in column 1
        new_tip='never'
    )
    pipette.drop_tip()

    # Step 2: Add 100 µL of media to wells in columns 2 to 10 and column 11
    pipette.pick_up_tip(tiprack_media.wells()[0])
    for col in range(2, 12):  # Columns 2 to 11
        pipette.transfer(
            media_volume,
            media_reservoir['A1'],
            plate.columns_by_name()[str(col)],  # Transfer to all wells in column 2 to 11
            new_tip='never'
        )
    pipette.drop_tip()

    # Step 3: Perform serial dilution from column 1 to column 10
    pipette.pick_up_tip(tiprack_serial.wells()[0])
    for col in range(1, 10):  # Columns 1 to 9 for serial dilution
        pipette.transfer(
            transfer_volume,
            plate.columns_by_name()[str(col)],  # Source column
            plate.columns_by_name()[str(col + 1)],  # Destination column
            mix_after=(3, 100),  # Mix well after transfer
            new_tip='never'  # Reuse the same tip
        )




    # Column 11 remains untouched at the end


Overwriting antifungal_testing.py


In [35]:
!opentrons_simulate antifungal_testing.py

/root/.opentrons/robot_settings.json not found. Loading defaults
from_conf None default {<InstrumentProbeType.PRIMARY: 1>: '/data/pressure_sensor_data.csv'}
Belt calibration not found.
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on slot B2
Logs from this command:
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
WARNING (instrument): The pipette p1000_multi_flex and its tip rack opentrons_96_tiprack_300ul appear to be mismatched. Please check your protocol.
Transferring 200.0 from A1 of NEST 12 Well Reservoir 15 mL on slot D2 to A1 of Corning 96 Well Plate 360 µL Flat on slot B1
	Asp

🌞 <font color='orange'>**Discussion Exercises**:</font>

1. What were the benefits of using ChatGPT for writing these protocols? What was it able to do succesfully and what did you have to do by hand?

2. Look at the entire protocol from the paper [video](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/bin/jove-44-2287-pmcvs_normal.mp4) again. What additional portions do you think you would be able to automate on the opentron? What parts could you automate with more advanced robotic hardware? What parts (if any) are do you think can't be reliably done using robotics?

3. Looking at the output of your simulations, can you propose a way in which you could have LLMs write protocols then use the simulation output to revise the protocol accordingly?

---
⭐ Double click to write down your answers here


```
Answer:

1. It can help me generate a brief architecture of the codes. It's fast and easy to use. I only need to focus on the specific parameters in most circumstances.

2.
  Parts of the Protocol That Can Be Automated with the Opentrons Robot:
    Aspirating medium and washing plates with PBS: After biofilm formation, careful aspiration of medium followed by PBS washes can also be handled with the Opentrons, with the right speed and volume settings to avoid disrupting biofilms. This may also require some fine-tuning based on experimental conditions, but it is a common automation task.
  Parts That Could Be Automated with More Advanced Robotic Hardware:
    Centrifugation and Cell Washing (Step 1):    The steps involving centrifugation and washing cells can be partially automated using robotic centrifuges or liquid-handling stations integrated with robotic arms for tube handling. However, the Opentrons itself would need to be integrated with such external hardware, as it doesn't handle centrifugation or vortexing directly.
    Mixing Cells and Counting: Automated systems for cell counting (e.g., integrated cell counters or imaging systems) could be used to automate the cell counting step before plating. However, this would require additional hardware beyond the standard Opentrons Flex configuration.
    Incubation with Controlled Conditions (Step 2): While the Opentrons can automate the liquid handling steps, incubating plates at specific temperatures and times could be managed by advanced robotic systems that integrate plate handling and incubation tasks. Some systems have robotic incubators that automatically take plates in and out.
  Parts That Are Difficult to Automate Reliably:
    Aseptic Techniques for Handling Cultures (Step 1): While basic liquid handling steps can be automated, ensuring aseptic conditions, particularly when handling live cultures, remains challenging. Robotic systems often struggle with maintaining sterile environments unless they are housed in a controlled environment such as a biosafety cabinet. Human oversight is still critical for aseptic techniques.
    Visual Inspection of Biofilm Formation: While automation could theoretically handle a lot of liquid handling and even imaging, confirming biofilm formation by visual inspection (e.g., using an inverted microscope) still requires human judgment for quality control, especially when determining the extent of biofilm formation.
    Handling Fragile Biofilms: Biofilms can be fragile and sensitive to disruption during medium exchanges. Automating these steps without disrupting the biofilm can be tricky, and a gentle approach would be necessary. Some advanced systems may be able to do this, but careful calibration and validation would be required.
3.  
    Generate Initial Protocol: The LLM creates an initial protocol based on inputs like the lab hardware, reagents, and the desired outcome.
    Simulate and Analyze: The Opentrons simulator runs the protocol and generates logs.
    Feedback Loop: The LLM reads the simulation logs and adjusts the protocol, flagging errors like pipette mismatches, calibration issues, or unexpected volumes.
    Re-run Simulations: Once corrections are applied, the revised protocol is re-simulated to check for further improvements.
    Iterative Revision: This process continues until the protocol runs error-free and meets predefined performance criteria.

```
---